In [1]:
from transformers import pipeline
import pandas as pd
from deep_translator import GoogleTranslator
import spacy
from spacy.cli import download
import os
import shutil
import random
import csv

2022-01-28 22:08:08.793674: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-28 22:08:08.793692: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
camembert_fill_mask = pipeline("fill-mask", model="camembert-base", tokenizer="camembert-base")

2022-01-28 22:08:13.806327: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-28 22:08:13.806389: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-28 22:08:13.806434: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sarahp-HP-ProBook-440-G7): /proc/driver/nvidia/version does not exist
2022-01-28 22:08:13.806960: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFCamembertForMaskedLM.

All the layers of

In [3]:
world_cities_df = pd.read_excel(r'worldcities.xlsx')

french_cities_csv = pd.read_csv('city_name.csv')
french_cities_csv.columns =['city']

french_cities = french_cities_csv.loc[world_cities_df['iso3'] == 'FRA']['city'].str.lower()

other_cities = world_cities_df.loc[world_cities_df['iso3'] != 'FRA']['city'].str.lower()

In [4]:
english_translator = GoogleTranslator(source='fr', target='en')
french_translator = GoogleTranslator(source='en', target='fr')

In [5]:
nlp = spacy.load('fr_core_news_sm')

In [6]:
f = open("augmented_data.csv", "w+")
f.close

<function TextIOWrapper.close()>

In [7]:
good_texts = [
    "Je voudrais un trajet de Paris vers Brest.",
    "Je veux aller de Nantes à Paris.",
    "Aide moi à aller de Montpellier à Lille",
    "Je veux faire Toulouse Bordeaux.",
    "Je veut arriver à nantes en partant de lyon.",
    "je voudrais aller de Paris à Nantes.",
    "Il me faudrait aller de Rennes à Bordeaux.",
    "Je dois faire le trajet de Brest à Strasbourg.",
    "Il faudrait que j'aille de Tours à Nesmy.",
    "J'aimerai faire Toulouse Nice.",
    "Comment aller de Marseille à Cannes?",
    "Il me faudrait un billet Angers Cahors.",
    "Comment aller à Nantes en partant de Rennes?",
    "Comment faire pour aller à Strasbourg depuis Toulouse?",
    "Comment se procurer un billet pour partir de Nice et aller à Brest?",
    "Comment voyager de Paris à Nantes?",
    "Trajet Brest Rennes",
    "Comment faire le trajet Challans Orvault?",
    "Combien de temps pour faire le voyage de Nice à Lyon?",
    "Je pars de Bordeaux et je vais à Calais"
]

In [8]:
bad_texts = [
    "Bonjour j'aime voyager.",
    "Je cherche à acheter une voiture.",
    "J'adore faire du bateau à Marseille.",
    "La route entre Nantes et Bordeaux est jolie.",
    "Je suis né à Rennes mais je vis maintenant à Angers.",
    "Quelle heure est-il?",
    "Je veux faire du vélo",
    "Je veux acheter un cadeau",
    "Je veux trouver un trajet"
]

In [9]:
def mask_texts(texts):
    masked_texts = []

    for text in texts:
        masked_texts.append(text)
        for index, word in enumerate(text.split()):
            s = text.split()
            s[index] = '<mask>'
            masked_text = ' '.join(s)
            masked_texts.append(masked_text)

    masked_texts = [s.lower() for s in masked_texts]        
    return masked_texts

In [10]:
def back_translation(text):
    english_text = english_translator.translate(text)
    french_text = french_translator.translate(english_text)

    return french_text

In [11]:
def augment_texts(texts, isValid):
    augmented_texts = []

    for masked_text in texts:
        if '<mask>' not in masked_text:
            augmented_texts.append(masked_text)
            augmented_texts.append(back_translation(masked_text))
        else:
            results = camembert_fill_mask(masked_text)
            for result in results:
                augmented_texts.append(result['sequence'])
                augmented_texts.append(back_translation(result['sequence']))
    augmented_texts = switch_cities(augmented_texts, isValid)
    #print(list(dict.fromkeys(augmented_texts)))
    return list(dict.fromkeys(augmented_texts))

In [12]:
def word_is_location(text, word):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'LOC' and ent.text == word:
            return True
    return False

In [13]:
def switch_cities(augmented_texts, isValid):
    augmented_texts_with_correct_cities = []

    for augmented_text in augmented_texts:
        new_text = augmented_text
        for index, word in enumerate(augmented_text.split()):
            word = word.replace(".", "")
            word = word.replace("?", "")
            word = word.replace(",", "")
            if isValid == "good" and word_is_location(augmented_text, word) and not french_cities.str.contains(word).any():
                s = new_text.split()
                s[index] = random.choice(list(french_cities))
                new_text = ' '.join(s)
            elif isValid == "bad" and word_is_location(augmented_text, word) and french_cities.str.contains(word).any():
                s = new_text.split()
                s[index] = random.choice(list(other_cities))
                new_text = ' '.join(s)
        augmented_texts_with_correct_cities.append(new_text)
    return augmented_texts_with_correct_cities

In [14]:
def data_augmentation(texts_to_augment, isValid):
    masked_texts = mask_texts(texts_to_augment)
    augmented_texts = augment_texts(masked_texts, isValid)

    with open('augmented_data.csv', 'a', newline='') as csvfile:
        fieldnames = ['sentence', 'valid']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        for index, augmented_text in enumerate(augmented_texts):
            print(augmented_text)
            writer.writerow({'sentence': augmented_text, 'valid':isValid=="good" and '1' or '0'})

In [15]:
data_augmentation(good_texts, "good")

je voudrais un trajet de cérons vers luc-en-diois
je voudrais un voyage de bons-en-chablais à baroncourt
Je voudrais un trajet de sains du thiaville vers brest.
Je voudrais un voyage de longpré-corps-saints à chambéry-chal.-les-eaux
je voudrais un trajet de bons-en-chablais vers longpré-corps-saints
je voudrais un voyage de donjeux à montaignac-st-hippolyte
baroncourt voudrais un trajet de romilly-la-puthenaye vers longpré-corps-saints
baroncourt voudrais un voyage de brion-laizy à sains du nord
J voudrais un trajet de thiaville vers tournemire-roquefort
Je voudrais un voyage de lamothe-landerron à romilly-la-puthenaye
longpré-corps-saints voudrais un trajet de tournemire-roquefort vers longueil-annel
Je voudrais un voyage de longpré-corps-saints à baroncourt
je cherche un trajet de chambéry-chal.-les-eaux vers chambéry-chal.-les-eaux
Je cherche un itinéraire de bons-en-chablais à lamothe-landerron
je recherche un trajet de luc-en-diois vers baroncourt
Je cherche un itinéraire de thiav

In [16]:
data_augmentation(bad_texts, "bad")

bonjour j'aime voyager.
Oui j'aime voyager.
Moi j'aime voyager.
Moi, j'aime voyager.
Et j'aime voyager.
Mais j'aime voyager.
Personnellement j'aime voyager.
bonjour adore voyager.
bonjour aime voyager.
bonjour pour voyager.
bonjour voyager.
bonjour aimant voyager.
bonjour et voyager.
bonjour et voyage.
bonjour j'aime beaucoup
bonjour j'aime bien
bonjour j'aime lire
bonjour j'aime aussi
bonjour j'aime bien aussi
bonjour j'aime tout
je cherche à acheter une voiture.
Je cherche à acheter une voiture.
Particulier cherche à acheter une voiture.
Femme cherche à acheter une voiture.
La femme cherche à acheter une voiture.
Madame cherche à acheter une voiture.
je recherche à acheter une voiture.
je songe à acheter une voiture.
Je pense acheter une voiture.
je pense à acheter une voiture.
je cherchais à acheter une voiture.
Je cherchais à acheter une voiture.
je cherche a acheter une voiture.
je cherche comment acheter une voiture.
Je cherche comment acheter une voiture.
je cherche ou acheter u